# Import

In [8]:
import numpy as np
import pandas as pd

rest1 = pd.read_csv('Rest_326.txt', sep=',', skiprows=4) # includes EEG, other channel information, timestamps, etc
rest2 = pd.read_csv('Rest_419.txt', sep=',', skiprows=4)
rest3 = pd.read_csv('Rest_425.txt', sep=',', skiprows=4)

rest1 = rest1.iloc[:, 1:9] # obtaining only EEG channel information
rest2 = rest2.iloc[:, 1:9]
rest3 = rest3.iloc[:, 1:9]

focus1 = pd.read_csv('Focus_326.txt', sep=',', skiprows=4) 
focus2 = pd.read_csv('Focus_425.txt', sep=',', skiprows=4)

focus1 = focus1.iloc[:, 1:9]
focus2 = focus2.iloc[:, 1:9]

rest = pd.concat([rest1, rest2, rest3], axis=0)
focus = pd.concat([focus1, focus2], axis=0)

datasets = [rest, focus]
datasets[0]

,EXG Channel 0,EXG Channel 1,EXG Channel 2,EXG Channel 3,EXG Channel 4,EXG Channel 5,EXG Channel 6,EXG Channel 7
0,-12336.330096,-9949.856693,-23118.185773,-34290.459369,-30306.909121,-27583.773742,-19681.761525,-20357.164187
1,-24862.046523,-20080.293129,-47057.574637,-65494.165181,-61139.732437,-57319.796958,-40135.932283,-42007.443846
2,-24356.695933,-19715.244438,-48294.073140,-72709.554161,-59682.041071,-55677.010796,-41374.956533,-42256.464631
3,-24911.108602,-20109.663321,-46624.531940,-64810.492374,-61298.519230,-57148.157912,-39733.220903,-41657.437880
4,-24310.114898,-19703.733290,-48672.488174,-73236.943571,-59556.692488,-55879.361138,-41744.833201,-42578.642676
...,...,...,...,...,...,...,...,...
37942,6629.415647,-24599.480581,-75301.887131,-82999.671459,-64449.265832,-60826.673904,-73478.141246,-68015.732529
37943,6816.298582,-24506.206752,-75225.868848,-85068.101891,-65879.486904,-61945.825749,-73822.805145,-68132.855670
37944,6625.235871,-24582.582662,-75158.232469,-83607.884778,-64548.373467,-60826.249221,-73379.212425,-67896.933007
37945,6794.527983,-24553.659505,-75402.537036,-84437.939160,-65783.329699,-61974.681851,-73947.282010,-68289.831971


# Filter

In [11]:
from scipy.signal import butter, filtfilt

def bandpass_filter(datasets):

    fs = 125 # sampling rate
    nyq = 0.5 * fs # Nyquist frequency
    lowcut = 0.5 / nyq # lowpass=0.5
    highcut = 50 / nyq #highpass=50 to prevent environmental electrical interference
    filtered = []
    
    for dataset in datasets:
        
        b, a = butter(1, [lowcut, highcut], btype='band')
        filter = filtfilt(b, a, dataset, padlen=7)
        
        filtered.append(filter)

    return filtered

f_datasets = bandpass_filter(datasets)
pd.DataFrame(f_datasets[0])

,0,1,2,3,4,5,6,7
0,10614.289888,12764.770403,662.934696,-8927.376607,-6433.465594,-1373.908093,5344.273616,5966.382935
1,22867.510510,26941.570114,3126.637504,-13513.719821,-10885.089381,-2722.061785,12051.495737,12950.883553
2,23442.253148,27738.254082,1035.467796,-19029.339874,-10378.947062,-531.445355,11149.230966,13003.906605
3,22378.225238,26457.384369,3113.854905,-13451.807681,-11444.132979,-3163.776093,11892.951159,12704.589054
4,23894.191593,28166.592061,1078.784368,-19003.093440,-9871.597611,-170.456467,11294.840885,13231.245225
...,...,...,...,...,...,...,...,...
114791,88707.093123,59282.557000,17244.241294,11136.713760,33395.164679,40425.421574,34495.203958,42883.871070
114792,88936.935265,59552.975544,17106.717448,9475.196902,31849.493745,39554.184545,34196.692636,42908.847721
114793,88503.370247,59149.121367,17045.800769,10455.348924,32960.882894,40237.365768,34322.226423,42745.732428
114794,89157.943528,59696.936631,17323.796897,10216.227522,32329.956785,39761.166282,34390.648040,43057.058473


# Epoching

In [15]:
def epoching(filtered_datasets):

    data = []
    for dataset in datasets:
        
        epoch_samples = 227 # amount of samples acquired in one burst of data from EEG 
        num_epochs = len(dataset) // epoch_samples
        
        epochs = []
        
        for i in range(num_epochs):
            start_index = i * epoch_samples
            end_index = start_index + epoch_samples
            epoch = dataset[start_index:end_index]
            epochs.append(epoch)

        data.append(epochs)
        
    return data
    
epoch_datasets = epoching(f_datasets)
pd.DataFrame(epoch_datasets[0][0])

,0,1,2,3,4,5,6,7
0,10614.289888,12764.770403,662.934696,-8927.376607,-6433.465594,-1373.908093,5344.273616,5966.382935
1,22867.510510,26941.570114,3126.637504,-13513.719821,-10885.089381,-2722.061785,12051.495737,12950.883553
2,23442.253148,27738.254082,1035.467796,-19029.339874,-10378.947062,-531.445355,11149.230966,13003.906605
3,22378.225238,26457.384369,3113.854905,-13451.807681,-11444.132979,-3163.776093,11892.951159,12704.589054
4,23894.191593,28166.592061,1078.784368,-19003.093440,-9871.597611,-170.456467,11294.840885,13231.245225
...,...,...,...,...,...,...,...,...
222,22482.835922,26511.846549,3153.909624,-13237.857742,-11281.846770,-3060.719360,11943.700944,12771.473681
223,23861.479493,28098.524969,1202.077440,-18549.920000,-9852.201528,-252.633311,11346.616264,13244.322714
224,22070.602684,26118.003490,3086.215273,-13319.413575,-11734.915865,-3369.809744,11792.420709,12560.382833
225,24238.750053,28462.675304,1307.211658,-18351.724314,-9413.836508,8.048435,11511.278629,13445.033293


# PSD Feature Extraction

In [18]:
from scipy.signal import welch
def psd_vals(epoch_datasets):

    psd_datasets = []
    
    for dataset in epoch_datasets:
        
        brain_waves = {'Delta': [0.5, 4], 'Theta': [4, 8], 'Alpha': [8, 12], 'Beta': [12, 30], 'Gamma': [30, 100]}
        psd_per_band = {}
        fs = 125
        count = 0
        
        for epoch in dataset:
            
            for band, (start, end) in brain_waves.items():
                f, psd = welch(epoch.T, fs=fs, nperseg=len(epoch))
                band_indices = np.where((f>=start) & (f<end))[0]
                band_psd = np.mean(np.mean(psd[:, band_indices], axis=1)) # averaging all psd values obtained from one epoch of data
                psd_per_band[count, band] = band_psd
        
            count += 1
        
        index = pd.MultiIndex.from_tuples(psd_per_band.keys())
        df = pd.DataFrame(psd_per_band.values(), index=index, columns=['Values']).unstack()
        psd_data = df['Values']

        psd_datasets.append(psd_data)
        
    return psd_datasets

psd_datasets = psd_vals(epoch_datasets)
psd_datasets[0]

,Alpha,Beta,Delta,Gamma,Theta
0,4.141130,2.132823,379.835892,46978.340222,7.178911
1,34.122017,9.481203,456.131891,43466.097877,81.991362
2,2.214825,1.737561,42.380732,43525.358751,8.794798
3,2.802052,2.231209,42.196767,43390.162979,10.668955
4,1.332171,2.526134,50.189892,43407.124757,10.121706
...,...,...,...,...,...
500,2.530607,1.020348,31.456339,32314.964702,8.367527
501,2.130624,0.960261,307.289512,32252.232165,3.721724
502,8.516488,1.525582,242.171283,30985.245930,27.583048
503,1.157527,0.845281,53.383732,32089.079781,11.660429


# Fourier Coefficient Feature Extraction

In [21]:
def fourier_coef(epoch_datasets):

    fc_data = []
    
    for epoch_data in epoch_datasets:
        
        fc = []
        
        for epoch in epoch_data:
        
            fourier_coefficients_channel = np.fft.fft(epoch, axis=0)
            fc.append(fourier_coefficients_channel)
        
        epoch_dfs = []

        for i, epoch_coeffs in enumerate(fc):

            epoch_df = pd.DataFrame(epoch_coeffs, columns=[f'Channel_{j}_Fourier_Coefficient' for j in range(epoch_coeffs.shape[1])])

            epoch_df['Epoch'] = i

            epoch_dfs.append(epoch_df)
        

        fourier_coefficients_df = pd.concat(epoch_dfs, ignore_index=True)
        

        mean_coefficients_per_epoch = fourier_coefficients_df.groupby('Epoch').mean()

        fc_data.append(np.real(mean_coefficients_per_epoch))

    return fc_data
    
fc_datasets = fourier_coef(epoch_datasets)
pd.DataFrame(fc_datasets[0])

,0,1,2,3,4,5,6,7
0,10614.289888,12764.770403,662.934696,-8927.376607,-6433.465594,-1373.908093,5344.273616,5966.382935
1,24520.872627,28726.258038,1456.833151,-18007.614937,-9074.925940,144.368702,11663.963039,13604.415275
2,24602.461862,28675.179061,2316.865709,-15732.879199,-8861.342635,-457.254880,12140.219724,13735.379801
3,23235.274406,27255.258962,3056.842762,-13469.206577,-10276.526176,-2238.418470,12137.770801,13146.971286
4,21597.948671,25633.577806,2884.406992,-13524.049626,-12107.372486,-3541.266990,11592.449858,12315.082015
...,...,...,...,...,...,...,...,...
500,88156.506952,58970.555374,16990.744385,10199.646646,32741.493178,40090.957848,34122.777208,42566.837996
501,87610.287093,58643.002158,16294.242563,7570.004045,30922.279439,39243.655448,33492.300761,42160.315530
502,88095.130346,59051.199011,16513.154546,7498.877841,30678.886023,39081.293688,33609.889584,42391.649778
503,88995.483949,59697.853708,17220.595645,9518.097233,31912.300659,39607.862129,34222.983556,42946.267544


# Merging

In [41]:
from sklearn.utils import shuffle

def conc(datasets):

    count = 0
    classes = []
    
    for data in datasets:

        y = pd.DataFrame(np.array(len(data) * [count]))
        classes.append(y)
        
        count += 1
        
    X = pd.concat([pd.DataFrame(datasets[0]), pd.DataFrame(datasets[1])], axis=0)
    y = pd.concat([classes[0], classes[1]], axis=0).rename(columns={0: 'Label'})
    #dataset = pd.concat([merge_X, merge_y], axis=1)
    
    return X, y

finished_dataset_psd, y_0 = conc(psd_datasets)
finished_dataset_fc, y_0 = conc(fc_datasets)

X_0 = pd.concat([finished_dataset_psd, finished_dataset_fc], axis=1)
finished_dataset = shuffle(pd.concat([X_0, y_0], axis=1))
finished_dataset.columns = finished_dataset.columns.astype(str)
X = finished_dataset.iloc[:, :13]
y = finished_dataset.iloc[:, 13]
X

,Alpha,Beta,Delta,Gamma,Theta,0,1,2,3,4,5,6,7
339,2.317615,1.352333,29.505063,37145.340119,9.563487,88676.742141,65872.628275,27879.504882,25582.175484,39902.273093,43989.239822,38072.588621,44208.570636
338,3.155898,0.781156,16.979217,37290.758454,6.938332,89468.861554,66519.046804,28447.228547,26440.576664,40043.042597,43936.025429,38389.191478,44653.341838
410,3.901247,0.777757,18.120008,33877.933847,6.416920,87061.291299,63990.368100,25065.122382,20701.985077,36644.952323,41852.770569,36474.158677,43195.068901
58,4.579897,1.633423,25.323126,30661.150360,19.372167,17805.970411,20275.421230,976.029912,-1654.693490,-1388.653200,1979.473289,10021.603155,11262.523399
383,2.240747,1.286500,63.773278,33955.522250,3.075306,87395.342215,64207.739615,25928.752719,21831.506289,36988.129428,42088.255171,36695.788328,43344.464132
...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,14.590907,3.757976,41.510001,4855.232640,5.212605,1029.732519,-4870.330765,-40898.320875,-37436.851558,-35751.553329,-15847.519114,-10720.847514,5657.099862
149,4.248405,1.778119,35.962677,37771.369803,5.356412,19151.287052,22392.886505,2527.806350,-3171.159200,-6599.393254,-1576.821721,11443.630411,12008.739565
312,5.127017,2.192523,116.290431,4817.290328,25.420899,2026.632042,-5182.215909,-41170.415736,-35746.952075,-34791.482144,-16573.768693,-10594.768103,6085.199947
187,2.066577,2.039067,28.607970,69608.964888,9.844421,103466.246262,77128.487567,34128.374139,33859.448762,50396.753155,55501.335407,47349.101029,51750.748323


# Classification

In [54]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pre_model = RandomForestClassifier()

param_grid = {
    'n_estimators': [50, 100, 150],  # Number of trees in the forest
    'max_features': ['sqrt', 'log2'],  # Number of features to consider at every split
    'max_depth': [None, 10, 20, 30],  # Maximum number of levels in each decision tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'bootstrap': [True, False]  # Method of selecting samples for training each tree
}

gs = GridSearchCV(pre_model, param_grid=param_grid, cv=5)
gs.fit(X_train, y_train)
best_params = gs.best_params_
print(best_params)

model = RandomForestClassifier(**best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)
test_acc = accuracy_score(y_test, y_pred)
train_acc = accuracy_score(y_train, y_pred_train)
test_class = classification_report(y_test, y_pred)
train_class = classification_report(y_train, y_pred_train)

print(test_class)
print(train_class)

{'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 50}
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       103
           1       1.00      1.00      1.00       102

    accuracy                           1.00       205
   macro avg       1.00      1.00      1.00       205
weighted avg       1.00      1.00      1.00       205

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       402
           1       1.00      1.00      1.00       418

    accuracy                           1.00       820
   macro avg       1.00      1.00      1.00       820
weighted avg       1.00      1.00      1.00       820

